In [2]:
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=55949b9db389cf175f573c4a91d09c05fb6c0def9dcbf5d1cf21f10efabbee31
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
# ! pip install py-vncorenlp

# Retrieval

In [4]:
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from sentence_transformers import InputExample, SentenceTransformer, losses
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
from transformers import DistilBertForQuestionAnswering
from transformers import DistilBertTokenizerFast

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
train_data = pd.read_json("/kaggle/input/e2eqa-wiki-zalo-ai/e2eqa-trainpublic_test-v1/e2eqa-train+public_test-v1/zac2022_train_merged_final.json")['data']
test_data = pd.read_json("/kaggle/input/e2eqa-wiki-zalo-ai/e2eqa-trainpublic_test-v1/e2eqa-train+public_test-v1/zac2022_testa_sample_submission.json")['data']

In [6]:
train_data[0]

{'id': '718d41cd997b2b44b0685ac54aa55bd8',
 'question': 'Thủ tướng Trung Quốc là gì',
 'title': 'Trung Quốc',
 'text': 'Thủ tướng Trung Quốc là nhân vật lãnh đạo chính phủ , chủ trì Quốc vụ viện gồm bốn phó thủ tướng cùng người đứng đầu các bộ và uỷ ban cấp bộ . Chủ tịch nước đương nhiệm là Tập Cận Bình , ông cũng là Tổng Bí thư của Đảng Cộng sản Trung Quốc và Chủ tịch Quân uỷ Trung Quốc , do vậy ông là lãnh đạo tối cao của Trung Quốc .',
 'category': 'PARTIAL_ANNOTATION',
 'is_long_answer': True}

In [7]:
train_data[1]

{'id': 'c926e7b0717202618a10dd907d4b4c39',
 'question': 'Đất nước nào không có quân đội',
 'title': '',
 'text': 'có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...',
 'short_candidate_start': 53,
 'short_candidate': 'Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...',
 'answer': 'wiki/Danh_sách_quốc_gia_không_có_lực_lượng_vũ_trang',
 'category': 'FULL_ANNOTATION',
 'is_long_answer': True}

In [8]:
all_id = []
train_questions = []
title = []
train_contexts = []
category = []
long_answer = []
short_candidate_start = []
short_candidate = []

for item in train_data:
    all_id.append(item['id'])
    train_questions.append(item['question'])
    title.append(item['title'])
    train_contexts.append(item['text'])
    category.append(item['category'])
    long_answer.append(1 if item['is_long_answer'] else 0)
    short_candidate_start.append(item['short_candidate_start'] if 'short_candidate_start' in item else None)
    short_candidate.append(item['short_candidate'] if 'short_candidate' in item else None)

In [9]:
df = pd.DataFrame()
df['id'] = all_id
df['context'] = train_contexts
df['title'] = title
df['question'] = train_questions
df['category'] = category
df['answer'] = long_answer
df['short_candidate_start'] = short_candidate_start
df['short_candidate'] = short_candidate

In [10]:
df.head()

,id,context,title,question,category,answer,short_candidate_start,short_candidate
0,718d41cd997b2b44b0685ac54aa55bd8,Thủ tướng Trung Quốc là nhân vật lãnh đạo chín...,Trung Quốc,Thủ tướng Trung Quốc là gì,PARTIAL_ANNOTATION,1,NaN,None
1,c926e7b0717202618a10dd907d4b4c39,"có 23 quốc gia không có lực lượng quân đội, ba...",,Đất nước nào không có quân đội,FULL_ANNOTATION,1,53.0,"Costa Rica, Iceland, Panama, Micronesia, Quần ..."
2,d38ef5bf1fb82b410026ed82c8a44cae,Raymondienne (hay Raymonde Dien) sinh ngày 13 ...,Raymondienne,Pháp tấn công xâm lược Việt Nam vào ngày tháng...,FALSE_LONG_ANSWER,0,NaN,None
3,b6b5589a98fdccd208dc752bac853993,Cúp cờ vua thế giới là tên gọi một số giải đấu...,Cúp cờ vua thế giới,Cờ vua còn có tên gọi nào khác,FALSE_LONG_ANSWER,0,NaN,None
4,82396a18fa9812bfec4d3ecb7ae60905,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,Shkhara,Núi nào cao nhất châu âu,FULL_ANNOTATION,1,73.0,núi Elbrus


In [11]:
retrieval_model = SentenceTransformer('keepitreal/vietnamese-sbert')

In [12]:
train_data = df

In [13]:
train_examples = []
for i in range(len(train_data)):
    train_examples.append(InputExample(texts=[train_data.iloc[i]['question'], train_data.iloc[i]['context']], label=float(train_data.iloc[i]['answer'])))

In [14]:
train_examples[0].texts

['Thủ tướng Trung Quốc là gì',
 'Thủ tướng Trung Quốc là nhân vật lãnh đạo chính phủ , chủ trì Quốc vụ viện gồm bốn phó thủ tướng cùng người đứng đầu các bộ và uỷ ban cấp bộ . Chủ tịch nước đương nhiệm là Tập Cận Bình , ông cũng là Tổng Bí thư của Đảng Cộng sản Trung Quốc và Chủ tịch Quân uỷ Trung Quốc , do vậy ông là lãnh đạo tối cao của Trung Quốc .']

In [15]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(retrieval_model)

In [16]:
retrieval_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1304 [00:00<?, ?it/s]

In [17]:
your_model_path = 'Retrievel_model'
retrieval_model.save(your_model_path)

In [18]:
passages = []
for i in range(len(df)):
    passages.append(df.iloc[i]['context'])
corpus_embeddings = retrieval_model.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/652 [00:00<?, ?it/s]

In [19]:
def search(query):
    print("Input question:", query)

    question_embedding = retrieval_model.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k =10)
    hits = hits[0]

    print("---------------------------")
    print("Top-10 retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:10]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

In [20]:
question = train_data.iloc[100].question

In [21]:
search(question)

Input question: nhạc sĩ nào là tác giả của bài hát nổi tiếng dáng đứng bến tre


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

---------------------------
Top-10 retrieval hits
	0.738	Hoàn cảnh ra đời . Nhạc sĩ Nguyễn Văn Tý nổi tiếng với nhiều ca khúc như : " Dư âm " ( 1949 ) , " Một khúc tâm tình của người Hà Tĩnh " ( 1974 ) , " Mẹ yêu con " , " Người đi xây hồ Kẻ Gỗ " ( 1976 ) , " Tấm áo chiến sĩ mẹ vá năm xưa " ( 1973 ) , " Dáng đứng Bến Tre " ( 1980 ) , " Cô đi nuôi dạy trẻ " ( 1980 ) .
	0.727	Từ hơn 500 năm về trước trong bài thơ " Lộ nhập Vân Đồn " , Nguyễn Trãi đã lần đầu tiên ca ngợi vịnh Hạ Long là " kỳ quan đá dựng giữa trời cao " .
	0.718	Hoa Lư là quê hương của nghệ thuật sân khấu chèo mà người sáng lập là bà Phạm Thị Trân , một vũ ca tài ba trong hoàng cung nhà Đinh . Đây là loại hình sân khấu được hình thành sớm nhất và tiêu biểu nhất của Việt Nam . Qua truyền thuyết về pháp sư Văn Du Tường thời Đinh dùng mưu diệt quỷ Xương Cuồng ở Bạch Hạc cho thấy nghệ thuật xiếc , tạp kỹ như đi trên dây , đánh đu , trồng cây chuối đã xuất hiện .
	0.709	Tóm tắt tác phẩm . Hồi thứ nhất . Vũ Như Tô , một kiến tr

# Rerank model

In [22]:
cross_encoder = CrossEncoder('keepitreal/vietnamese-sbert', num_labels=1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at keepitreal/vietnamese-sbert and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
cross_encoder.fit(train_dataloader=train_dataloader, epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1304 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [24]:
cross_encoder.save('Rerank_model')

# Reading Comprehension

In [25]:
index = 13
text1 = df.iloc[index].context
text2 = df.iloc[index].question
print(text2)
print(text1)
score = cross_encoder.predict([text1, text2])
print(f"Similar score: {score}")

Chiếc cầu rộng nhất thế giới
Cầu có chiều dài 483 m, rộng 7 m gồm 11 nhịp với kinh phí đầu tư 104 tỷ đồng. Công ty xây dựng công trình giao thông 61 tiếp tục trải bê-tông nhựa để kịp thông xe kỹ thuật vào ngày 26 tháng 4 năm ??. Cầu Bình Phước thuộc Dự án đường xuyên Á, thiết kế cho 4 làn xe ô tô và 2 làn xe thô sơ, tải trọng 30 tấn. Hiện nay đã có cầu Bình Phước 2 nằm song song với cầu Bình Phước cũ, tải trọng từ 25 - 30 tấn.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similar score: 0.7752692103385925


In [26]:
df.head()

,id,context,title,question,category,answer,short_candidate_start,short_candidate
0,718d41cd997b2b44b0685ac54aa55bd8,Thủ tướng Trung Quốc là nhân vật lãnh đạo chín...,Trung Quốc,Thủ tướng Trung Quốc là gì,PARTIAL_ANNOTATION,1,NaN,None
1,c926e7b0717202618a10dd907d4b4c39,"có 23 quốc gia không có lực lượng quân đội, ba...",,Đất nước nào không có quân đội,FULL_ANNOTATION,1,53.0,"Costa Rica, Iceland, Panama, Micronesia, Quần ..."
2,d38ef5bf1fb82b410026ed82c8a44cae,Raymondienne (hay Raymonde Dien) sinh ngày 13 ...,Raymondienne,Pháp tấn công xâm lược Việt Nam vào ngày tháng...,FALSE_LONG_ANSWER,0,NaN,None
3,b6b5589a98fdccd208dc752bac853993,Cúp cờ vua thế giới là tên gọi một số giải đấu...,Cúp cờ vua thế giới,Cờ vua còn có tên gọi nào khác,FALSE_LONG_ANSWER,0,NaN,None
4,82396a18fa9812bfec4d3ecb7ae60905,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,Shkhara,Núi nào cao nhất châu âu,FULL_ANNOTATION,1,73.0,núi Elbrus


In [27]:
df_qa = df[df['category'] == 'FULL_ANNOTATION']

In [28]:
df_qa

,id,context,title,question,category,answer,short_candidate_start,short_candidate
1,c926e7b0717202618a10dd907d4b4c39,"có 23 quốc gia không có lực lượng quân đội, ba...",,Đất nước nào không có quân đội,FULL_ANNOTATION,1,53.0,"Costa Rica, Iceland, Panama, Micronesia, Quần ..."
4,82396a18fa9812bfec4d3ecb7ae60905,Đỉnh núi nằm ở phần trung tâm của dãy núi Đại ...,Shkhara,Núi nào cao nhất châu âu,FULL_ANNOTATION,1,73.0,núi Elbrus
9,ee8a272bcefac7b96037a1571984b1f8,"Lịch sử . Ai Cập bị La Mã chiếm năm 30 TCN , v...",Alexandria,Thành phố nào là thủ phủ của Ai Cập trong đế q...,FULL_ANNOTATION,1,48.0,Alexandria
16,c51604b25a641b2665c4ce1784b8cc6a,Các nghiên cứu lịch sử cho thấy Cổ Am rất có t...,Cổ Am,Ai là người đứng đầu trong cuộc chống lại chín...,FULL_ANNOTATION,1,65.0,Lê Chân
23,04bf774a424133d27c34895960937b87,"Phục Hưng (tiếng Pháp: ""Renaissance"", , , từ ""...",Phục Hưng,Nền văn hoá Phục Hưng bắt nguồn từ nước nào,FULL_ANNOTATION,1,205.0,Ý
...,...,...,...,...,...,...,...,...
20826,be7f71e3927632f3f3f2afa68ff1b91b,Tiền giấy xuất hiện . Năm 1396 thời Trần Thuận...,Tiền tệ Đại Việt thời Trần,vua nào cho phát hành tiền giấy đầu tiên ở việ...,FULL_ANNOTATION,1,54.0,Hồ Quý Ly
20836,5769cee0487a9674306d271854244b39,Máy quay phim: Phát minh kỳ diệu của anh em Lu...,,Ai là người phát minh ra máy quay phim,FULL_ANNOTATION,1,37.0,anh em Lumière Phương Hiền
20840,e47529f7ade3caaaa60096137798c6ff,Ngũ Hành Sơn hay núi Non Nước là một danh thắn...,Ngũ Hành Sơn,tên quận nào ở đà nẵng cũng là tên một danh th...,FULL_ANNOTATION,1,285.0,quận Ngũ Hành Sơn
20842,935572f75515f8d8ab831648589899c6,Trận Như Nguyệt là một trận đánh lớn diễn ra ở...,Trận Như Nguyệt,trận đánh nào có ý nghĩa quyết định trong khán...,FULL_ANNOTATION,1,0.0,Trận Như Nguyệt


In [29]:
contexts = []
questions = []
answers = []
for i in range(df_qa.shape[0]):
    line = df_qa.iloc[i]
    answer = {}
    contexts.append(line['context'])
    questions.append(line['question'])
    answer['answer_start'] = int(line['short_candidate_start'])
    answer['text'] = line['short_candidate']
    answers.append(answer)

In [30]:
print(len(contexts))
print(len(questions))
print(len(answers))

4989
4989
4989


In [31]:
print(contexts[0], end='\n\n')
print(questions[0], end='\n\n')
print(answers[0], end='\n\n')

có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...

Đất nước nào không có quân đội

{'answer_start': 53, 'text': 'Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...'}



In [32]:
def add_end_idx(answers, contexts):
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    # this means the answer is off by 'n' tokens
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n

In [33]:
add_end_idx(answers, contexts)

In [34]:
print(contexts[0], end='\n\n')
print(questions[0], end='\n\n')
print(answers[0], end='\n\n')

có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...

Đất nước nào không có quân đội

{'answer_start': 53, 'text': 'Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...', 'answer_end': 126}



In [35]:
from transformers import XLMRobertaTokenizerFast, XLMRobertaForQuestionAnswering

tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
model = XLMRobertaForQuestionAnswering.from_pretrained("xlm-roberta-base")

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
train_encodings = tokenizer(contexts, questions, truncation=True, padding=True, return_offsets_mapping=True)

In [37]:
print("Tokens:", train_encodings["input_ids"][0])
print("Attention Mask:", train_encodings["attention_mask"][0])

Tokens: [0, 524, 1105, 10895, 3529, 687, 524, 9611, 6372, 29225, 25738, 4, 8609, 33256, 12, 23047, 92051, 4, 114749, 4, 149674, 4, 37992, 86, 3478, 4, 10022, 249994, 19, 54520, 157530, 4, 544, 103302, 27, 2, 2, 137229, 3042, 3941, 687, 524, 29225, 25738, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [38]:
# In ra chiều dài của các thành phần
print("Number of input_ids:", len(train_encodings["input_ids"]))
print("Number of attention_mask:", len(train_encodings["attention_mask"]))

Number of input_ids: 4989
Number of attention_mask: 4989


In [39]:
len(train_encodings["input_ids"][0])

512

In [40]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift one token forward
        go_back = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end']-go_back)
            go_back +=1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, answers)



In [41]:
print("Tokens:", train_encodings["input_ids"][0])
print("Attention Mask:", train_encodings["attention_mask"][0])
print("Start Positions:", train_encodings["start_positions"][0])
print("End Positions:", train_encodings["end_positions"][0])

Tokens: [0, 524, 1105, 10895, 3529, 687, 524, 9611, 6372, 29225, 25738, 4, 8609, 33256, 12, 23047, 92051, 4, 114749, 4, 149674, 4, 37992, 86, 3478, 4, 10022, 249994, 19, 54520, 157530, 4, 544, 103302, 27, 2, 2, 137229, 3042, 3941, 687, 524, 29225, 25738, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [42]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
# val_dataset = SquadDataset(val_encodings)

In [43]:
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=5e-5)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(3):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 2: 100%|██████████| 312/312 [04:33<00:00,  1.14it/s, loss=0.17] 


In [44]:
model_path = 'QA/models'
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('QA/models/tokenizer_config.json',
 'QA/models/special_tokens_map.json',
 'QA/models/sentencepiece.bpe.model',
 'QA/models/added_tokens.json',
 'QA/models/tokenizer.json')

In [45]:
# Chắc chắn rằng cả model và tensors đều ở trên cùng một thiết bị (GPU)
import torch.nn.functional as F

model.to('cuda:0')

i = 101

context = contexts[i]
question = questions[i]
true_answers = answers[i]['text']

# Mã hóa context và question
encoding = tokenizer(context, question, return_tensors="pt")

# Di chuyển encoding lên GPU
encoding = {key: value.to('cuda:0') for key, value in encoding.items()}

# Dự đoán
with torch.no_grad():
    output = model(**encoding)

# Lấy vị trí có xác suất lớn nhất
start_probs = F.softmax(output['start_logits'], dim=1)
end_probs = F.softmax(output['end_logits'], dim=1)

# Lấy vị trí có xác suất lớn nhất
start_index = torch.argmax(start_probs)
end_index = torch.argmax(end_probs) + 1

# Lấy độ tin cậy
confidence = start_probs[0][start_index].item() * end_probs[0][end_index-1].item()

# Lấy câu trả lời dự đoán
answer = tokenizer.decode(encoding["input_ids"][0][start_index:end_index])

print(f"Question: {question}")
print(f"Context: {context}")
print(f"True Answer: {true_answers}")
print("Predicted Answer:", answer)
print(f"Confidence: {round(confidence*100, 2)}%")


Question: nhất gia bán thiên hạ là lời khen tặng của vua tự đức dành cho người dân làng nào
Context: Truyền thống khoa bảng . Làng được vua Tự Đức ban tặng lời vàng : " " Mộ Trạch nhất gia bán thiên hạ " " ( Mộ Trạch tài năng bằng nửa thiên hạ ) .
True Answer: Mộ Trạch
Predicted Answer: Mộ Trạch
Confidence: 99.28%


In [72]:
def search_context(query):
#     print("Input question:", query)

    question_embedding = retrieval_model.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k = 200)
    hits = hits[0]
        
    cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
#     print(hits)
#     for i in range(10):
#         print(f"Score: {round(hits[i]['cross-score'], 2)}  - {passages[hits[i]['corpus_id']]}") 
#         print()
    return passages[hits[0]['corpus_id']]

def search_answer(question, context):
    encoding = tokenizer(context, question, return_tensors="pt")
    encoding = {key: value.to('cuda:0') for key, value in encoding.items()}
    with torch.no_grad():
        output = model(**encoding)

    # Lấy vị trí có xác suất lớn nhất
    start_probs = F.softmax(output['start_logits'], dim=1)
    end_probs = F.softmax(output['end_logits'], dim=1)

    # Lấy vị trí có xác suất lớn nhất
    start_index = torch.argmax(start_probs)
    end_index = torch.argmax(end_probs) + 1
    answer = tokenizer.decode(encoding["input_ids"][0][start_index:end_index])
    
    print(f"Question: {question}")
    print(f"Context: {context}")
#     print(f"True Answer: {true_answers}")
    print("Predicted Answer:", answer)
#     print(f"Confidence: {round(confidence*100, 2)}%")


In [76]:
id=1
context = search_context(test_data[id]['question'])
search_answer(test_data[id]['question'], context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Question: Tổng thống Hoa Kỳ thứ 45 là ai
Context: Barack Hussein Obama II ( IPA : ; sinh ngày 4 tháng 8 năm 1961 ) là tổng thống thứ 44 của Hoa Kỳ từ năm 2009 đến năm 2017 . Ông là người Mỹ gốc Phi đầu tiên được bầu vào chức vụ này . Lớn lên ở Honolulu , Hawaii , cá nhân ông thừa hưởng những nền văn hoá Phi-Âu-Á-Mỹ của thế giới từ thuở thiếu thời , Obama tốt nghiệp Viện Đại học Columbia và Trường Luật Viện Đại học Harvard , nơi ông từng là chủ tịch " Harvard Law Review " .
Predicted Answer: Barack Hussein Obama II
